# Object Detection API Demo

<table align="left"><td>
  <a target="_blank"  href="https://colab.sandbox.google.com/github/tensorflow/models/blob/master/research/object_detection/colab_tutorials/object_detection_tutorial.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab
  </a>
</td><td>
  <a target="_blank"  href="https://github.com/tensorflow/models/blob/master/research/object_detection/colab_tutorials/object_detection_tutorial.ipynb">
    <img width=32px src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
</td></table>

Welcome to the [Object Detection API](https://github.com/tensorflow/models/tree/master/research/object_detection). This notebook will walk you step by step through the process of using a pre-trained model to detect objects in an image.

> **Important**: This tutorial is to help you through the first step towards using [Object Detection API](https://github.com/tensorflow/models/tree/master/research/object_detection) to build models. If you just just need an off the shelf model that does the job, see the [TFHub object detection example](https://colab.sandbox.google.com/github/tensorflow/hub/blob/master/examples/colab/object_detection.ipynb).

# Setup

Important: If you're running on a local machine, be sure to follow the [installation instructions](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/installation.md). This notebook includes only what's necessary to run in Colab.

### Install

In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import PIL
# For downloading the image.
import matplotlib.pyplot as plt
import tempfile
from six.moves.urllib.request import urlopen
from six import BytesIO
from PIL import ImageOps
# For drawing onto the image.
import numpy as np
from PIL import Image
from PIL import ImageColor
from PIL import ImageDraw
from PIL import ImageFont
from PIL import ImageOps
# For measuring the inference time.
import time
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import base64

Get `tensorflow/models` or `cd` to parent directory of the repository.

In [2]:
cd ../..

C:\Users\Sabrina\Test\models\research


Compile protobufs and install the object_detection package

### Imports

In [3]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
from IPython.display import display
import pathlib

Import the object detection module.

In [4]:
from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

In [5]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
if len(physical_devices) > 0:
       tf.config.experimental.set_memory_growth(physical_devices[0], True)

Patches:

In [5]:
cd ../../..

C:\Users\Sabrina


# Model preparation 

## Variables

Any model exported using the `export_inference_graph.py` tool can be loaded here simply by changing the path.

By default we use an "SSD with Mobilenet" model here. See the [detection model zoo](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md) for a list of other models that can be run out-of-the-box with varying speeds and accuracies.

## Loader

In [6]:
def load_model(model_dir):
    

    #model_dir = pathlib.Path(model_dir)/"saved_model"
    model_dir = pathlib.Path(model_dir)
    model = tf.saved_model.load(str(model_dir))

    return model

## Loading label map
Label maps map indices to category names, so that when our convolution network predicts `5`, we know that this corresponds to `airplane`.  Here we use internal utility functions, but anything that returns a dictionary mapping integers to appropriate string labels would be fine

In [7]:
# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = './Test/models/workplace/training_demo/annotations/label_map.pbtxt'
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)
print(category_index)

{1: {'id': 1, 'name': 'bref'}, 2: {'id': 2, 'name': 'bref-3-en-1'}, 3: {'id': 3, 'name': 'bref-linge'}, 4: {'id': 4, 'name': 'bref-linge-oxy'}, 5: {'id': 5, 'name': 'bref-nettoyant'}, 6: {'id': 6, 'name': 'bref-power-active'}, 7: {'id': 7, 'name': 'chat-linge-noir-fonce'}, 8: {'id': 8, 'name': 'chat-poudre-haute-performance'}, 9: {'id': 9, 'name': 'chat-poudre-machine'}, 10: {'id': 10, 'name': 'chat-poudre-main'}, 11: {'id': 11, 'name': 'chat-poudre-universel'}, 12: {'id': 12, 'name': 'chat-power-gel'}, 13: {'id': 13, 'name': 'chat-power-gel-main'}, 14: {'id': 14, 'name': 'chat-premium-gel'}, 15: {'id': 15, 'name': 'chat-savon-de-marseille'}, 16: {'id': 16, 'name': 'isis-gel-machine'}, 17: {'id': 17, 'name': 'isis-gel-main'}, 18: {'id': 18, 'name': 'isis-poudre-machine'}, 19: {'id': 19, 'name': 'isis-poudre-main'}, 20: {'id': 20, 'name': 'pril-isis-5-en-1'}, 21: {'id': 21, 'name': 'pril-isis-gold'}, 22: {'id': 22, 'name': 'pril-isis-javel-power'}, 23: {'id': 23, 'name': 'pril-isis-peau

# Detection

Load an object detection model:

In [8]:
model_dir = './Test/models/workplace/training_demo/exported-models/saved_model'
detection_model = load_model(model_dir)

Check the model's input signature, it expects a batch of 3-color images of type uint8:

In [9]:
print(detection_model.signatures['serving_default'])

ConcreteFunction signature_wrapper(*, input_tensor)
  Args:
    input_tensor: uint8 Tensor, shape=(1, None, None, 3)
  Returns:
    {'detection_anchor_indices': <1>, 'detection_boxes': <2>, 'detection_classes': <3>, 'detection_multiclass_scores': <4>, 'detection_scores': <5>, 'num_detections': <6>, 'raw_detection_boxes': <7>, 'raw_detection_scores': <8>}
      <1>: float32 Tensor, shape=(1, 100)
      <2>: float32 Tensor, shape=(1, 100, 4)
      <3>: float32 Tensor, shape=(1, 100)
      <4>: float32 Tensor, shape=(1, 100, 24)
      <5>: float32 Tensor, shape=(1, 100)
      <6>: float32 Tensor, shape=(1,)
      <7>: float32 Tensor, shape=(1, 51150, 4)
      <8>: float32 Tensor, shape=(1, 51150, 24)


And returns several outputs:

In [10]:
detection_model.signatures['serving_default'].output_dtypes

{'detection_anchor_indices': tf.float32,
 'detection_boxes': tf.float32,
 'detection_classes': tf.float32,
 'detection_multiclass_scores': tf.float32,
 'detection_scores': tf.float32,
 'num_detections': tf.float32,
 'raw_detection_boxes': tf.float32,
 'raw_detection_scores': tf.float32}

In [11]:
detection_model.signatures['serving_default'].output_shapes

{'detection_anchor_indices': TensorShape([1, 100]),
 'detection_boxes': TensorShape([1, 100, 4]),
 'detection_classes': TensorShape([1, 100]),
 'detection_multiclass_scores': TensorShape([1, 100, 24]),
 'detection_scores': TensorShape([1, 100]),
 'num_detections': TensorShape([1]),
 'raw_detection_boxes': TensorShape([1, 51150, 4]),
 'raw_detection_scores': TensorShape([1, 51150, 24])}

In [12]:
def load_img(path):
    img = tf.io.read_file(path)
    img = tf.image.decode_jpeg(img, channels=3)
    return img

In [13]:
def display_image(image):
    fig = plt.figure(figsize=(100, 100))
    plt.grid(False)
    plt.imshow(image)
    plt.savefig('image_product_detection.jpg')

In [14]:
def draw_bounding_box_on_image(image,
                               ymin,
                               xmin,
                               ymax,
                               xmax,
                               color,
                               font,
                               thickness=4,
                               display_str_list=()):
  
        draw = ImageDraw.Draw(image)
        im_width, im_height = image.size
        (left, right, top, bottom) = (xmin * im_width, xmax * im_width,ymin * im_height, ymax * im_height)
        draw.line([(left, top), (left, bottom), (right, bottom), (right, top),
                 (left, top)],
                width=thickness,
                fill=color)
        display_str_heights = [font.getsize(ds)[1] for ds in display_str_list]
        total_display_str_height = (1 + 2 * 0.05) * sum(display_str_heights)

        if top > total_display_str_height:
            text_bottom = top
        else:
            text_bottom = top + total_display_str_height
          # Reverse list and print from bottom to top.
        for display_str in display_str_list[::-1]:
            text_width, text_height = font.getsize(display_str)
            margin = np.ceil(0.05 * text_height)
            draw.rectangle([(left, text_bottom - text_height - 2 * margin),
                            (left + text_width, text_bottom)],
                           fill=color)
            draw.text((left + margin, text_bottom - text_height - margin),
                      display_str,
                      fill="black",
                      font=font)
            text_bottom -= text_height - 2 * margin

In [97]:
def draw_boxes(image, boxes, class_names, scores, max_boxes=550000):
    
    colors = list(ImageColor.colormap.values())
    #colors =['#00FFFF'',#FFA500','#FF0000', '#1E90FF', '#FFFF00', '#32CD32']


    try:
        font = ImageFont.truetype("/usr/share/fonts/truetype/liberation/LiberationSansNarrow-Regular.ttf",
                                  25)
    except IOError:
        font = ImageFont.load_default()

        for i in range(min(boxes.shape[0], max_boxes)):
                if(scores[i]>0.4):
                    ymin, xmin, ymax, xmax = tuple(boxes[i])
                    display_str = "{}: {}%".format(class_names[i].decode("ascii"),int(100 * scores[i]))
                    color = colors[hash(class_names[i]) % len(colors)]
                    image_pil = Image.fromarray(np.uint8(image)).convert("RGB")
                    draw_bounding_box_on_image(
                          image_pil,
                          ymin,
                          xmin,
                          ymax,
                          xmax,
                          color,
                          font,
                          display_str_list=[display_str])
                    np.copyto(image, np.array(image_pil))
    return image

Add a wrapper function to call the model, and cleanup the outputs:

In [98]:
def run_inference_for_single_image(model, image):
    image = np.asarray(image)
    # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
    input_tensor = tf.convert_to_tensor(image)
    # The model expects a batch of images, so add an axis with `tf.newaxis`.
    input_tensor = input_tensor[tf.newaxis,...]

    # Run inference
    model_fn = model.signatures['serving_default']
    output_dict = model_fn(input_tensor)

    # All outputs are batches tensors.
    # Convert to numpy arrays, and take index [0] to remove the batch dimension.
    # We're only interested in the first num_detections.
    num_detections = int(output_dict.pop('num_detections'))
    output_dict = {key:value[0, :num_detections].numpy() 
                     for key,value in output_dict.items()}
    output_dict['num_detections'] = num_detections
   
    # detection_classes should be ints.
    output_dict['detection_classes'] = output_dict['detection_classes'].astype(np.int64)
    
    # Handle models with masks:
    if 'detection_masks' in output_dict:
        # Reframe the the bbox mask to the image size.
        detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
                  output_dict['detection_masks'], output_dict['detection_boxes'],
                   image.shape[0], image.shape[1])      
        detection_masks_reframed = tf.cast(detection_masks_reframed > 0.5,
                                           tf.uint8)
        output_dict['detection_masks_reframed'] = detection_masks_reframed.numpy()

    return output_dict

Run it on each test image and show the results:

In [99]:
def show_inference(model, image_path):
    result={}
    img = load_img(image_path)
    # the array based representation of the image will be used later in order to prepare the
    # result image with boxes and labels on it.
    image_np = np.array(Image.open(image_path))
    # Actual detection.
    output_dict = run_inference_for_single_image(model, image_np)
    print(output_dict['detection_classes'])
    # Visualization of the results of a detection.
    #vis_util.visualize_boxes_and_labels_on_image_array(
          #image_np,
          #output_dict['detection_boxes'],
          #output_dict['detection_classes'],
          #output_dict['detection_scores'],
          #category_index,
          #instance_masks=output_dict.get('detection_masks_reframed', None),
          #use_normalized_coordinates=True,
          #line_thickness=8)

    #display(Image.fromarray(image_np))
    result["detection_boxes"]=output_dict['detection_boxes']
    result["detection_class_entities"]=[str(category_index[classe]['name']).encode() for classe in output_dict['detection_classes']]
    result["detection_scores"]=output_dict['detection_scores']
    image_with_boxes = draw_boxes(
          img.numpy(), result["detection_boxes"],
          result["detection_class_entities"], result["detection_scores"])

    display_image(image_with_boxes)

In [124]:
show_inference(detection_model, "image_viber_2021-04-14_15-15-13.jpg")


[ 9  9  9  9  9  9  9 11 11  9  9  9 11  8  3  9  8 11 16 11  9 16 23 11
  9  8 15 15  9  6 16  9  9 11 11  9  9 11  9  9  8 10  9  9  9  9  9 22
 15  9  9  9  9 18  9 14 22  9  7 11  7  8 10  8  9 16 15  9 18 10  9  9
 11 20  9  9  9 15 11 11  9  9 15  8  9 15  1  9  5 15  6 11  9  9 15 10
  9 10  7 16]


MemoryError: 

## Instance Segmentation

In [ ]:
model_name = "mask_rcnn_inception_resnet_v2_atrous_coco_2018_01_28"
masking_model = load_model(model_name)

The instance segmentation model includes a `detection_masks` output:

In [ ]:
masking_model.output_shapes

In [ ]:
for image_path in TEST_IMAGE_PATHS:
      show_inference(masking_model, image_path)

In [26]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'    # Suppress TensorFlow logging (1)
import pathlib
import tensorflow as tf

tf.get_logger().setLevel('ERROR')           # Suppress TensorFlow logging (2)

# Enable GPU dynamic memory allocation
gpus = tf.config.experimental.list_physical_devices('GPU')
print(gpus)

[]


In [1]:
import tensorflow as tf

In [72]:
hello = tf.constant('Hello, TensorFlow!')
sess = tf.Session()

In [5]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


In [3]:
from tensorflow.python.client import device_lib

def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU']

In [4]:
get_available_gpus()

['/device:GPU:0']

In [2]:
from tensorflow.python.client import device_lib 
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 15809207687243814364
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 17605329079755107218
physical_device_desc: "device: XLA_CPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 1391427175
locality {
  bus_id: 1
  links {
  }
}
incarnation: 10700582968850507108
physical_device_desc: "device: 0, name: GeForce GTX 1050, pci bus id: 0000:01:00.0, compute capability: 6.1"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 12027032967906934487
physical_device_desc: "device: XLA_GPU device"
]


In [7]:
gpus = tf.config.experimental.list_physical_devices('GPU')
print(gpus)
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
              tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
1 Physical GPUs, 1 Logical GPUs
